In [1]:
# 3-dimensional embeddings of the sequence “Your journey starts with one step”

import torch
# inputs with initial random 3 dimension embeddings 
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your (x^1)
   [0.55, 0.87, 0.66], # journey (x^2)
   [0.57, 0.85, 0.64], # starts (x^3)
   [0.22, 0.58, 0.33], # with (x^4)   
   [0.77, 0.25, 0.10], # one (x^5)   
   [0.05, 0.80, 0.55]] # step (x^6)   
)

# attention scores for 2nd input ("journey") wrt to all other inputs - using dot products
# mutliplies element wise i.e. 1st ele with 1st ele, 2nd with 2nd and so on
query = inputs[1]                           
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

# normalize the attention scores to convert them to attention weights
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

# computing z(2) by multiplying the embedded input tokens, x(i), with the corresponding attention weights and then summing the resulting vectors
query = inputs[1]        
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)



tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)
tensor([0.4419, 0.6515, 0.5683])


In [2]:
attn_scores = inputs @ inputs.T
attn_weights = torch.softmax(attn_scores, dim=-1)

all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [3]:
# now with trainable weights
# we compute 3 matrices: query Wq (for current input), key Wk, value Wv
# taking 2nd input as query
x_2 = inputs[1]    
d_in = inputs.shape[1]     
d_out = 2
# Note that in GPT-like models, the input and output dimensions are usually the same, but to better follow the computation, we’ll use different input (d_in=3) and output (d_out=2) dimensions here.

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) 
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) 
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

query_2 = x_2 @ W_query
keys = inputs @ W_key 
values = inputs @ W_value

print(d_in)
print(x_2, W_query.shape, W_query)
print(query_2)
print(keys)


3
tensor([0.5500, 0.8700, 0.6600]) torch.Size([3, 2]) Parameter containing:
tensor([[0.8660, 0.3234],
        [0.1195, 0.7528],
        [0.1735, 0.7190]])
tensor([0.6947, 1.3074])
tensor([[0.9625, 0.5235],
        [0.9628, 1.1608],
        [0.9521, 1.1397],
        [0.4820, 0.7065],
        [0.4925, 0.4377],
        [0.6229, 0.9499]])


In [4]:
attn_scores_2 = query_2 @ keys.T      
print(attn_scores_2)

d_k = keys.shape[-1] 
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1) 
print(attn_weights_2)

context_vec_2 = attn_weights_2 @ values 
print(context_vec_2)

tensor([1.3531, 2.1864, 2.1514, 1.2585, 0.9144, 1.6747])
tensor([0.1336, 0.2408, 0.2349, 0.1250, 0.0980, 0.1677])
tensor([0.9041, 1.1978])


In [5]:
import torch.nn as nn 
class SelfAttention_v1(nn.Module):    
    def __init__(self, d_in, d_out): 
        super().__init__() 
        self.W_query = nn.Parameter(torch.rand(d_in, d_out)) 
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out)) 
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x): 
        keys = x @ self.W_key 
        queries = x @ self.W_query 
        values = x @ self.W_value 
        attn_scores = queries @ keys.T # omega 
        attn_weights = torch.softmax( attn_scores / keys.shape[-1]**0.5, dim=-1 ) 
        context_vec = attn_weights @ values 
        return context_vec
    
torch.manual_seed(123) 
sa_v1 = SelfAttention_v1(d_in, d_out) 
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [6]:
# using nn.Linear

class SelfAttention_v2(nn.Module):    
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__() 
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias) 
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias) 
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x): 
        keys = self.W_key(x) 
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T # omega 
        attn_weights = torch.softmax( attn_scores / keys.shape[-1]**0.5, dim=-1 ) 
        context_vec = attn_weights @ values 
        return context_vec
    
torch.manual_seed(789) 
sa_v2 = SelfAttention_v2(d_in, d_out) 
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [7]:
queries = sa_v2.W_query(inputs)    
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T 
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1) 
print("attn weights without causal attn:\n", attn_weights)

context_length = attn_scores.shape[0] 
mask_simple = torch.tril(torch.ones(context_length, context_length)) 
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1) 
print("\nsimple mask: (needs normal normalization, since e^0 = 1, but we need it to be 0)\n", mask_simple, "\n\nmask:(softmax can normalize with this after converting 1s into -inf since e^-inf = 0)\n", mask)

masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print("masked attention scores:\n" , masked)

attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1) 
print("attn weights:\n", attn_weights)

attn weights without causal attn:
 tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

simple mask: (needs normal normalization, since e^0 = 1, but we need it to be 0)
 tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]]) 

mask:(softmax can normalize with this after converting 1s into -inf since e^-inf = 0)
 tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])

In [8]:
# implementing it in a class
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        keys = self.W_key(x)
        # print(keys.shape)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

In [9]:
torch.manual_seed(123) 
dropout = torch.nn.Dropout(0.5) # half the values are removed to lessen overfitting
print(dropout(attn_weights))

batch = torch.stack((inputs, inputs), dim=0) 
print(batch.shape)

torch.manual_seed(123) 
context_length = batch.shape[1] 
ca = CausalAttention(d_in, d_out, context_length, 0.0) 
context_vecs = ca(batch) 
print("context vec:\n", context_vecs)
print("\ncontext_vecs.shape:", context_vecs.shape)

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)
torch.Size([2, 6, 3])
context vec:
 tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)

context_vecs.shape: torch.Size([2, 6, 2])


In [10]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length,
        dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(
                d_in, d_out, context_length, dropout, qkv_bias
            ) 
        for _ in range(num_heads)]
    )
        
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [11]:
torch.manual_seed(123)
context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2

# layering 2 causal attention modules, each have d_out 2, which is concatenated to give 4
mha = MultiHeadAttentionWrapper(
   d_in, d_out, context_length, 0.0, num_heads=2
)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


In [16]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        print("attn_weights shape:", attn_weights.shape)
        print("values shape:", values.shape)
        print("context_vec shape:", context_vec.shape)
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        print("context_vec shape:", context_vec.shape)
        context_vec = self.out_proj(context_vec) # optional projection
        print("context_vec shape:", context_vec.shape)

        return context_vec

In [17]:
torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
print(batch.shape)
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

torch.Size([2, 6, 3])
attn_weights shape: torch.Size([2, 2, 6, 6])
values shape: torch.Size([2, 2, 6, 1])
context_vec shape: torch.Size([2, 6, 2, 1])
context_vec shape: torch.Size([2, 6, 2])
context_vec shape: torch.Size([2, 6, 2])
tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
